# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import vactionPy_apiKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "data/weather_df.csv"

weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
0,Vaini,-21.2000,-175.2000,66.200,100,20,1.03
1,Hilo,19.7297,-155.0900,66.524,68,75,3.60
2,Mareeba,-17.0000,145.4333,75.524,83,90,2.57
3,Hūn,29.1268,15.9477,78.080,24,68,6.05
4,Hay River,60.8156,-115.7999,19.418,86,100,3.60
...,...,...,...,...,...,...,...
562,Desna,50.9273,30.7605,33.656,100,100,2.00
563,São Raimundo Nonato,-9.0153,-42.6994,96.422,25,65,4.47
564,Aberdeen,57.1437,-2.0981,38.138,60,40,3.09
565,City of San Pedro,14.3500,121.0167,81.122,74,75,2.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=vactionPy_apiKey)

In [5]:
# Store 'Latitude' and 'Longitude' as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

# 'Humidity' as float data type
humidity = weather_df["Humidity"].astype(float)

In [6]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
hotel_df = weather_df.loc[(weather_df['Temperature']>= 65)
                           & (weather_df['Temperature']< 80)
                            & (weather_df['Humidity']>= 10)
                           & (weather_df['Humidity']< 20)]
hotel_df.dropna(how='any')

hotel_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
139,Creel,27.7500,-107.6333,68.504,16,100,5.31
188,Bāfq,31.6128,55.4107,73.616,12,1,2.07
294,Dārāb,28.7519,54.5444,69.134,11,0,1.21
365,Concepción de Buenos Aires,19.9667,-103.2667,78.800,14,1,3.09
408,Najrān,17.4924,44.1277,77.000,15,0,2.06
436,Pahrump,36.2083,-115.9839,74.318,12,47,1.34
493,León,21.1167,-101.6667,78.800,14,1,3.60
509,Sirsa,29.5333,75.0167,74.642,16,0,3.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-8-cb2d5672db59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH),Hotel Name
139,Creel,27.7500,-107.6333,68.504,16,100,5.31,
188,Bāfq,31.6128,55.4107,73.616,12,1,2.07,
294,Dārāb,28.7519,54.5444,69.134,11,0,1.21,
365,Concepción de Buenos Aires,19.9667,-103.2667,78.800,14,1,3.09,
408,Najrān,17.4924,44.1277,77.000,15,0,2.06,
436,Pahrump,36.2083,-115.9839,74.318,12,47,1.34,
493,León,21.1167,-101.6667,78.800,14,1,3.60,
509,Sirsa,29.5333,75.0167,74.642,16,0,3.52,


In [9]:
# Google Places - hotel finder

hotel_list = []

for index, city in hotel_df.iterrows():
    #print(city[1], city[2])
    lat = city[1]
    lng = city[2]
    target_search = "lodging"
    params = {
    "location": f"{lat},{lng}",
    "types": target_search,
    "rankby": "distance",
    "key": vactionPy_apiKey
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    hotel_response = response.json()
    hotel_name = hotel_response["results"][0]["name"]
    hotel_list.append(hotel_name)

hotel_df['Hotel Name'] = hotel_df.append(hotel_list, ignore_index=True)

<ipython-input-9-a4def43a1480>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_df.append(hotel_list, ignore_index=True)


In [10]:
hotel_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH),Hotel Name
139,Creel,27.7500,-107.6333,68.504,16,100,5.31,NaN
188,Bāfq,31.6128,55.4107,73.616,12,1,2.07,NaN
294,Dārāb,28.7519,54.5444,69.134,11,0,1.21,NaN
365,Concepción de Buenos Aires,19.9667,-103.2667,78.800,14,1,3.09,NaN
408,Najrān,17.4924,44.1277,77.000,15,0,2.06,NaN
436,Pahrump,36.2083,-115.9839,74.318,12,47,1.34,NaN
493,León,21.1167,-101.6667,78.800,14,1,3.60,NaN
509,Sirsa,29.5333,75.0167,74.642,16,0,3.52,NaN


In [ ]:
# Google Places code
params = {
    "address": my_address,
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

print("Drill #5: A Dentist Near My House")

my_geo = requests.get(base_url, params).json()

lat = my_geo["results"][0]["geometry"]["location"]["lat"]
lng = my_geo["results"][0]["geometry"]["location"]["lng"]


# Use lat, lng to use places API to find nearest dentist
target_search = "dentist"

params = {
    "location": f"{lat},{lng}",
    "types": target_search,
    "rankby": "distance",
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Run request
response = requests.get(base_url, params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)
my_dentist = response.json()

# Print the JSON (pretty printed)
# print(json.dumps(my_dentist, indent=4, sort_keys=True))

# Print the name and address of the first dentist that appears
print(my_dentist["results"][0]["name"])
print(my_dentist["results"][0]["vicinity"])

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
